# Load LLM

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
)

# Chatbot that uses pages and summaries

In [23]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser

def combine_documents(documents: list[Document]) -> str:
    return "\n\n".join([document.page_content for document in documents])


REPHRASE_SYSTEM_PROMPT = """
<PERSONA>
Eres un especialista resolviendo dudas sobre libros de ficción
</PERSONA>

<TASK>
Tu tarea es refrasear la solicitud del usuario para genera una solicitud refraseada.

- Puedes corregir los errores gramaticales
- Puedes mejorar la semántica y orden léxico de la palabras para un mejor entendimiento
</TASK>
"""

REPHRASE_USER_PROMPT = """{user_request}"""

rephrase_prompt = ChatPromptTemplate([
    SystemMessage(content=REPHRASE_SYSTEM_PROMPT),
    ("user", REPHRASE_USER_PROMPT)
])


QA_SYSTEM_PROMPT = """
<PERSONA>
Eres un especialista resolviendo dudas sobre libros de ficción
</PERSONA>

<TAREA>
Tu tarea es responder la pregunta del usuario.
</TAREA>

<RESTRICCIONES>
- Solo responde la pregunta del usuario tomando como contexto lo provisto en <CONTEXTO>.
</RESTRICCIONES>

<CONTEXTO 1>
{context_1}
</CONTEXTO 1>

<CONTEXTO 2>
{context_2}
</CONTEXTO 2>
"""

QA_USER_PROMPT = """
user question: {user_request}
rephrased user question: {rephrased_request}
"""

qa_prompt = ChatPromptTemplate([
    ("system", QA_SYSTEM_PROMPT),
    ("user", QA_USER_PROMPT)
])

url = "https://e7f4684c-fd33-4db0-b1d3-268870ecb84d.europe-west3-0.gcp.cloud.qdrant.io:6333"
api_key = os.getenv("QDRANT_API_KEY")

client = QdrantClient(
    url=url,
    api_key=api_key,
    https=True,
    timeout=300
)

vector_store_page = QdrantVectorStore(
    client=client,
    collection_name="db-book-page",
    embedding=OpenAIEmbeddings(model="text-embedding-ada-002"),
)

vector_store_summarized = QdrantVectorStore(
    client=client,
    collection_name="db-book-summarized",
    embedding=OpenAIEmbeddings(model="text-embedding-ada-002"),
)

# def debug(x):
#     print(x)
#     return x
#    | RunnableLambda(debug)


simple_chatbot = (
    {
        "user_request": itemgetter("user_request"),
        "rephrased_request": rephrase_prompt | llm | StrOutputParser()
    }
    | RunnablePassthrough() 
    | {
        "user_request": itemgetter("user_request"),
        "rephrased_request": itemgetter("rephrased_request"),
        "context_1": itemgetter("rephrased_request") | vector_store_page.as_retriever(search_kwargs={"k": 5}) | RunnableLambda(combine_documents),
        "context_2": itemgetter("rephrased_request") | vector_store_summarized.as_retriever(search_kwargs={"k": 5}) | RunnableLambda(combine_documents)
    }
    | qa_prompt 
    | llm
    | StrOutputParser()
)

In [24]:
simple_chatbot.invoke({"user_request": "quien es susan fletcher?"})

'Susan Fletcher es una criptógrafa estrella que trabaja en la Agencia de Seguridad Nacional (NSA) en la Sección de Criptografía. Es descrita como una mujer inteligente y atractiva, con un alto coeficiente intelectual. A lo largo de la historia, se enfrenta a situaciones críticas y peligrosas, tanto en su trabajo en la NSA como en su vida personal, especialmente en relación con su prometido, David Becker. Susan es un personaje central en la trama, involucrada en la resolución de un código indescifrable y en la búsqueda de una clave de acceso crucial.'

# Chatbot that uses pages, summaries and neighbors

In [25]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/fortaleza-digital.pdf")
pages = loader.load()

filtered_documents = [page for page in pages if len(page.page_content) > 0]

In [37]:
filtered_documents[101].page_content

'enamorada de un profesor universitario que trabajaba como un esclavo por un\nsueldo miserable. Sería una pena que Susan malgastara su herencia genética\nsuperior procreando con un degenerado, sobre todo pudiendo hacerlo con Greg.\nTendríamos unos hijos perfectos\n, pensó.\n—¿En qué estás trabajando? —preguntó Hale, cambiando de táctica.\nSusan no dijo nada.\n—Menuda compañera \nestás\n hecha. ¿De veras que no puedo echar un vistazo?\nSe levantó y empezó a rodear el círculo de terminales en dirección a ella.\nSusan presintió que la curiosidad de Hale podía causar graves problemas.\nTomó una repentina decisión.\n—Es un diagnóstico —explicó, aprovechando la mentira del comandante.\nEl paró en seco.\n—¿Un diagnóstico? —Parecía dudoso—. ¿Dedicas un sábado a realizar un\ndiagnóstico, en lugar de jugar con el profe?\n—Se llama David.\n—Da igual.\nSusan le fulminó con la mirada.\n—¿No tienes nada mejor que hacer?\n—¿Intentas deshacerte de mí?\n—Pues la verdad es que sí.\n—Caramba, Sue, eso me

In [45]:
def extract_pages(documents: list[Document]) -> list[int]:
    pages = []

    for doc in documents:
        #print(doc.metadata)
        pages.append(doc.metadata["page"])

    return sorted(list(set(pages)))

def add_neighbors(pages: list[int]) -> list[int]:
    n = 3
    pages_with_neighbors = []
    
    for p in pages:
        index_start = max(p - n, 0)
        index_end = min(p + n, 355)
    
        pages_with_neighbors.extend(list(range(index_start, index_end)))
    
    return sorted(list(set(pages_with_neighbors)))

def get_context_by_number_of_page(pages: list[int]) -> str:
    contexts = []

    for p in pages:
        contexts.append(filtered_documents[p])
    
    #print(len(contexts))
    return combine_documents(contexts)
    

simple_chatbot = (
    {
        "user_request": itemgetter("user_request"),
        "rephrased_request": rephrase_prompt | llm | StrOutputParser()
    }
    | RunnablePassthrough() 
    | {
        "user_request": itemgetter("user_request"),
        "rephrased_request": itemgetter("rephrased_request"),
        "pages": itemgetter("rephrased_request") | vector_store_page.as_retriever(search_kwargs={"k": 5}) | RunnableLambda(extract_pages) | RunnableLambda(add_neighbors),
    }
    | RunnablePassthrough()
    | {
        "user_request": itemgetter("user_request"),
        "rephrased_request": itemgetter("rephrased_request"),
        "context_1": itemgetter("pages") | RunnableLambda(get_context_by_number_of_page),
        "context_2": itemgetter("rephrased_request") | vector_store_summarized.as_retriever(search_kwargs={"k": 5}) | RunnableLambda(combine_documents)
    }
    | qa_prompt 
    | llm
    | RunnableLambda(lambda x: x.content)
)

In [47]:
simple_chatbot.invoke({"user_request": "quien es susan fletcher?"})

'Susan Fletcher es una criptógrafa estrella que trabaja en la Agencia de Seguridad Nacional (NSA) en la Sección de Criptografía. Es una mujer inteligente y atractiva, con un alto coeficiente intelectual, y desempeña un papel crucial en la trama de "La fortaleza digital" de Dan Brown. A lo largo de la novela, se enfrenta a desafíos relacionados con un algoritmo indescifrable y se ve envuelta en situaciones de intriga y peligro. Además, tiene una conexión personal e intelectual con David Becker, un profesor universitario especializado en idiomas extranjeros.'

In [30]:
[2, 104, 173, 219, 301]

[2, 104, 173, 219, 301]

In [34]:
len([0,
  1,
  2,
  3,
  4,
  101,
  102,
  103,
  104,
  105,
  106,
  170,
  171,
  172,
  173,
  174,
  175,
  216,
  217,
  218,
  219,
  220,
  221,
  298,
  299,
  300,
  301,
  302,
  303])

29

# Let's add a friendly answer

In [50]:
from langchain_core.output_parsers.string import StrOutputParser

FRIENDLY_PROMPT = """
Convierte la respuesta provista por el usuario en una respuesta amigable que contenga emojis

user_question: {user_request}
answer: {answer}

friendly answer: 
"""

friendly_prompt = ChatPromptTemplate({("user", FRIENDLY_PROMPT)})

simple_chatbot = (
    {
        "user_request": itemgetter("user_request"),
        "rephrased_request": rephrase_prompt | llm | StrOutputParser(),
    }
    | RunnablePassthrough()
    | {
        "user_request": itemgetter("user_request"),
        "rephrased_request": itemgetter("rephrased_request"),
        "pages": (
            itemgetter("rephrased_request")
            | vector_store_page.as_retriever(search_kwargs={"k": 5})
            | RunnableLambda(extract_pages)
            | RunnableLambda(add_neighbors)
        )
    }
    | RunnablePassthrough()
    | {
        "user_request": itemgetter("user_request"),
        "answer": {
            "user_request": itemgetter("user_request"),
            "rephrased_request": itemgetter("rephrased_request"),
            "context_1": (
                itemgetter("pages")
                | RunnableLambda(get_context_by_number_of_page)
            ),
            "context_2": (
                itemgetter("rephrased_request")
                | vector_store_summarized.as_retriever(search_kwargs={"k": 5})
                | RunnableLambda(combine_documents)
            )
        }
        | qa_prompt
        | llm
        | StrOutputParser(),
    }
    | friendly_prompt
    | llm
    | StrOutputParser()
)


In [51]:
simple_chatbot.invoke({"user_request": "quien es susan fletcher?"})

'¡Claro! 😊 Susan Fletcher es una brillante criptógrafa que trabaja en la Agencia de Seguridad Nacional (NSA) en la Sección de Criptografía. 🕵️\u200d♀️🔐 Es una mujer súper inteligente y atractiva, con un coeficiente intelectual altísimo. 📚✨ Susan juega un papel crucial en la emocionante trama de "La fortaleza digital" de Dan Brown. 📖🔍 Se enfrenta a un complicado problema con un algoritmo indescifrable y vive situaciones llenas de intriga y peligro en su trabajo. 😮💻 Además, tiene una conexión especial tanto personal como intelectual con David Becker, un profesor universitario experto en idiomas extranjeros. 💑🌍'

# Let's generate a step back question

...